In [1]:
import numpy as np
import pandas as pd
from astropy.coordinates import SkyCoord
from astropy import units as u
import glob

In [2]:
# Loop over all files

filenames = glob.glob("/project2/chihway/raulteixeira/data/bdf_photometry/dr3_1_1_bdf_0000**.h5")

In [3]:
len(filenames)

49

In [4]:
def get_dataframes(filenames):
    all_dataframes = []
    for filename in filenames:
        # Open catalogs
        all_dataframes.append(pd.read_hdf(filename, key='data/table', mode='r'))

    data_df = pd.concat(all_dataframes)
    
    return data_df

In [5]:
# Open catalogs
data_df = get_dataframes(filenames[:1])

specz = pd.read_table("/project2/chihway/delve_shear/BRPORTAL_E_6315_18670.csv", sep=',')

In [7]:
data_df.columns

Index(['index', 'COADD_OBJECT_ID', 'RA', 'DEC', 'BDF_MAG_G', 'BDF_MAG_R',
       'BDF_MAG_I', 'BDF_MAG_Z', 'BDF_MAG_ERR_G', 'BDF_MAG_ERR_R',
       'BDF_MAG_ERR_I', 'BDF_MAG_ERR_Z', 'BDF_S2N', 'BDF_T', 'BDF_T_ERR',
       'BDF_T_RATIO', 'BDF_FLUX_G', 'BDF_FLUX_R', 'BDF_FLUX_I', 'BDF_FLUX_Z',
       'BDF_FLUX_ERR_G', 'BDF_FLUX_ERR_R', 'BDF_FLUX_ERR_I', 'BDF_FLUX_ERR_Z',
       'TILENAME'],
      dtype='object')

In [8]:
specz.columns

Index(['IDSPEC', 'RA', 'DEC', 'Z', 'ERR_Z', 'FLAG', 'SOURCE', 'CLASS',
       'FLAG_DES', 'ID'],
      dtype='object')

In [5]:
# Match spectra to DELVE catalog
spec_cd = SkyCoord(ra=specz['RA'].values*u.degree, dec=specz['DEC'].values*u.degree)
delve_cd = SkyCoord(ra=data_df['RA'].values*u.deg, dec=data_df['DEC'].values*u.deg)
idx, d2d, d3d = delve_cd.match_to_catalog_sky(spec_cd)
good_matches = d2d < 1.0*u.arcsec

In [6]:
print(len(np.unique(idx[good_matches])), np.count_nonzero(good_matches))

523813 524026


In [7]:
# Add spectra to DELVE catalog
data_df['zspec'] = np.nan
data_df.loc[good_matches, 'zspec'] = specz.iloc[idx[good_matches], specz.columns.get_loc('Z')].values

In [8]:
# Save each tile separately
unique_tilenames = np.unique(data_df.TILENAME)
unique_tilenames.shape

(6684,)

In [ ]:
for tilename in unique_tilenames:
    data_df_sub = data_df[data_df.TILENAME==tilename]
    data_df_sub.to_csv('/home/raulteixeira/scratch-midway2/CosmicShearData/spectiles/'+tilename+'.csv.gz', sep=',', header=True, index=False)

In [1]:
# Save one file containing the subset with matched spectra.
data_df_spec = data_df[np.isfinite(data_df.zspec.values)]
data_df_spec.to_csv('/home/raulteixeira/scratch-midway2/CosmicShearData/DELVE_data_with_zspec012623.csv.gz', sep=',', header=True, index=False)